In [2]:
!pip install cvxpygen

     |████████████████████████████████| 394 kB 25.4 MB/s 
     |████████████████████████████████| 2.8 MB 43.7 MB/s 
     |████████████████████████████████| 211 kB 49.7 MB/s 
  Created wheel for cvxpygen: filename=cvxpygen-0.1.5-py3-none-any.whl size=432204 sha256=c94872d93f749726142f904d55e190ab16d1fc0fa94abed52e83fcdd83367684
  Stored in directory: /root/.cache/pip/wheels/92/f3/d6/984f01198540de4c69250d23175fc57a328735bdc19356ab12
Successfully built cvxpygen
  Attempting uninstall: cvxpy
    Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


In [1]:
import cvxpy as cp

m, n = 3, 2
x = cp.Variable(n, name='x')
A = cp.Parameter((m, n), name='A', sparsity=[(0, 0), (0, 1), (1, 1)])
b = cp.Parameter(m, name='b')
problem = cp.Problem(cp.Minimize(cp.sum_squares(A @ x - b)), [x >= 0])

In [2]:
import numpy as np

np.random.seed(0)
A.value = np.zeros((m, n))
A.value[0, 0] = np.random.randn()
A.value[0, 1] = np.random.randn()
A.value[1, 1] = np.random.randn()
b.value = np.random.randn(m)
problem.solve()

0.9550720544957328

Create C code for the solver

In [3]:
from cvxpygen import cpg

cpg.generate_code(problem, code_dir='nonneg_LS', solver='SCS')

Generating code with CVXPYgen ...
CVXPYgen finished generating code.
Compiling python wrapper with CVXPYgen ... 
CVXPYgen finished compiling python wrapper.


Compare Python wrapper for C execution with conventional method

In [4]:
import time
import sys

# import extension module and register custom CVXPY solve method
from nonneg_LS.cpg_solver import cpg_solve
problem.register_solve('cpg', cpg_solve)

# solve problem conventionally
t0 = time.time()
val = problem.solve(solver='SCS')
t1 = time.time()
sys.stdout.write('\nCVXPY\nSolve time: %.3f ms\n' % (1000*(t1-t0)))
sys.stdout.write('Primal solution: x = [%.6f, %.6f]\n' % tuple(x.value))
sys.stdout.write('Dual solution: d0 = [%.6f, %.6f]\n' % tuple(problem.constraints[0].dual_value))
sys.stdout.write('Objective function value: %.6f\n' % val)

# solve problem with C code via python wrapper
t0 = time.time()
val = problem.solve(method='cpg', updated_params=['A', 'b'], verbose=False)
t1 = time.time()
sys.stdout.write('\nCVXPYgen\nSolve time: %.3f ms\n' % (1000 * (t1 - t0)))
sys.stdout.write('Primal solution: x = [%.6f, %.6f]\n' % tuple(x.value))
sys.stdout.write('Dual solution: d0 = [%.6f, %.6f]\n' % tuple(problem.constraints[0].dual_value))
sys.stdout.write('Objective function value: %.6f\n' % val)


CVXPY
Solve time: 20.535 ms
Primal solution: x = [0.837470, 1.908129]
Dual solution: d0 = [0.000000, 0.000000]
Objective function value: 0.955072

CVXPYgen
Solve time: 1.445 ms
Primal solution: x = [0.837470, 1.908129]
Dual solution: d0 = [0.000000, 0.000000]
Objective function value: 0.955054


In [ ]:
# !bash

bash: cannot set terminal process group (602): Inappropriate ioctl for device
bash: no job control in this shell
/content# cd nonneg_LS/c/build
/content/nonneg_LS/c/build# cmake ..
-- Single precision floats (32bit) are OFF
-- Long integers (64bit) are OFF
-- COMPILER_OPTS = -DUSE_LAPACK -DCTRLC
-- Configuring done
-- Generating done
-- Build files have been written to: /content/nonneg_LS/c/build
/content/nonneg_LS/c/build# cmake --build . --target cpg_example
Scanning dependencies of target cpg_example
[  3%] Building C object CMakeFiles/cpg_example.dir/src/cpg_workspace.c.o
[  6%] Building C object CMakeFiles/cpg_example.dir/src/cpg_solve.c.o
[ 10%] Building C object CMakeFiles/cpg_example.dir/solver_code/src/aa.c.o
[ 13%] Building C object CMakeFiles/cpg_example.dir/solver_code/src/cones.c.o
[ 16%] Building C object CMakeFiles/cpg_example.dir/solver_code/src/ctrlc.c.o
[ 20%] Building C object CMakeFiles/cpg_example.dir/solver_code/src/linalg.c.o
[ 23%] Building C object CMakeFiles/c